# List and patterns

Do you remember how to declare lists as literals? really easy, right?!

In [1]:
let items = [1;2;3;4]

val items : int list = [1; 2; 3; 4]


We can use this with pattern matching, if we remember how to do concatenation of lists:

In [4]:
"a" :: []

- : string list = ["a"]


In [5]:
let rec sum l =
  match l with
  | [] -> 0
  | hd :: tl -> hd + sum tl
;;

val sum : int list -> int = <fun>


This function is _not tail recursive_, but converting it to a tail recursive function is easy:

In [7]:
let sum l =
  let rec sum' total = function
  | [] -> total
  | hd :: tl -> sum' (hd + total) tl in
  sum' 0 l
;;

val sum : int list -> int = <fun>


A good example of usage of List functions is this, taken from the book:

In [19]:
(* We create a list of max length of a given header list and row list *)
(* Notice the usage of List.map and List.fold *)
(* List.map2_exn takes two lists and a function to combine them, if the lists are not the same size it throws an exception *)
let max_widths header rows = 
  let lengths l = List.map ~f:String.length l in
  List.fold rows ~init:(lengths header) ~f:(fun acc row -> List.map2_exn ~f:Int.max acc (lengths rows))
;;

(* It takes a string and pad it so it fix the expected length *)
let pad str length =
  " " ^ String.make (length - String.length str + 1) ' '
;;

(* It renders a row, padding it *)
let render_row row widths =
  let padded = List.map2_exn ~f:pad row widths in
  "|" ^ String.concat ~sep:"|" padded ^ "|"
;;

let render_separator widths =
  let separators = List.map widths ~f:(fun w -> String.make (w + 2) '-') in
  "|" ^ String.concat ~sep:"+" separators ^ "|"
;;

let render_table header rows =
  let widths = max_widths header rows in
  String.concat ~sep:"\n" 
      (render_row header widths :: render_separator widths :: List.map rows ~f:(fun x -> render_row rows widths))
;;

printf "%s\n" (render_table ["language";"architect";"first release"] [["Lisp";"John McCarthy";"1958"];["C";"Dennis Ritchie";"1969"];["ML";"Robin Milner";"1973"];["OCaml";"Xavier Leroy";"1996"]])

val max_widths :
  Core.String.t Core.List.t ->
  Core.String.t Core.List.t -> Core.Int.t Core.List.t = <fun>


val pad : Core.String.t -> int -> string = <fun>


val render_row : Core.String.t Core.List.t -> int Core.List.t -> string =
  <fun>


val render_separator : int Core.List.t -> string = <fun>


val render_table :
  Core.String.t Core.List.t -> Core.String.t Core.List.t -> Core.String.t =
  <fun>


error: compile_error

### Terser patterns:

Consider the following function:

In [20]:
let rec destutter l =
  match l with
  | [] -> []
  | [hd] -> [hd]
  | hd :: hd' :: tl -> if hd = hd' then destutter (hd' :: tl) else hd :: destutter (hd' :: tl)
;;

val destutter : 'a list -> 'a list = <fun>


This will work, but it looks ugly and it is not tail recursive, let's start by changing the function a little so it is clearer and easier to understand:

In [21]:
let rec destutter = function
  | [] | [_] as l -> l
  | hd :: (hd' :: _ as tl) -> if hd = hd' then destutter tl else hd :: destutter tl
;;

val destutter : 'a list -> 'a list = <fun>


This is a lot clearer and shorter, we can use the keyword `when` to make it even easier to read and get rid of that `if...else`

In [22]:
let rec destutter = function
  | [] |[_] as l -> l
  | hd :: (hd' :: _ as tl) when hd = hd' -> destutter tl
  | hd :: tl -> hd :: destutter tl
;;

val destutter : 'a list -> 'a list = <fun>


Be careful with `when` clauses, it doesn't help the compiler to know when a pattern is exaustive or not. For example:

In [23]:
let rec count_some l =
  match l with
  | [] -> 0
  | x :: tl when Option.is_none x -> count_some tl
  | x :: tl when Option.is_some x -> 1 + count_some tl
;;

File "[23]", line 2, characters 2-132:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
_::_
(However, some guarded clause may match this value.)


val count_some : 'a Core.Option.t list -> int = <fun>


See the compiler warning? even if we already saw that pattern is already covered... This is because the compiler cannot check we have all the patterns covered in the `when` clause. It will be much better to change it to this:

In [27]:
let rec count_some l =
  match l with
  | [] -> 0
  | x :: tl when Option.is_none x -> count_some tl
  | _ :: tl -> 1 + count_some tl
;;

val count_some : 'a Core.Option.t list -> int = <fun>


The right answer to this question is using the explicit pattern, because we know there are only three options:

In [28]:
let rec count_some l =
  match l with
  | [] -> 0
  | None :: tl -> count_some tl
  | Some _ :: tl -> 1 + count_some tl
;;

val count_some : 'a option list -> int = <fun>
